# T-RES using DeezyMatch with REL disambiguation

REL disambiguation **filtering out microtoponyms** and without adding place of publication.

In [ ]:
import os
import sys
import sqlite3
from pathlib import Path

from t_res.geoparser import pipeline, ranking, linking

In [ ]:
# --------------------------------------
# Instantiate the ranker:
myranker = ranking.Ranker(
    method="deezymatch",
    resources_path="../resources/wikidata/",
    strvar_parameters=dict(),
    deezy_parameters={
        # Paths and filenames of DeezyMatch models and data:
        "dm_path": str(Path("../resources/deezymatch/").resolve()),
        "dm_cands": "wkdtalts",
        "dm_model": "w2v_ocr",
        "dm_output": "deezymatch_on_the_fly",
        # Ranking measures:
        "ranking_metric": "faiss",
        "selection_threshold": 50,
        "num_candidates": 1,
        "verbose": False,
        # DeezyMatch training:
        "overwrite_training": False,
        "do_test": False,
    },
)

In [ ]:
with sqlite3.connect("../resources/rel_db/embeddings_database.db") as conn:
    cursor = conn.cursor()
    mylinker = linking.Linker(
        method="reldisamb",
        resources_path="../resources/",
        rel_params={
            "model_path": "../resources/models/disambiguation/",
            "data_path": "../experiments/outputs/data/lwm/",
            "training_split": "originalsplit",
            "db_embeddings": cursor,
            "with_publication": False,
            "without_microtoponyms": True,
            "do_test": False,
            "default_publname": "",
            "default_publwqid": "",
        },
        overwrite_training=False,
    )

In [ ]:
geoparser = pipeline.Pipeline(myranker=myranker, mylinker=mylinker)

In [ ]:
resolved = geoparser.run_text("A remarkable case of rattening has just occurred in the building trade next to the Market-street of Shefrield, but also in Lancaster. Not in Nottingham though. Not in Ashton either, nor in Salop!")
    
for r in resolved:
    print(r)

In [ ]:
resolved = geoparser.run_sentence("A remarkable case of rattening has just occurred in the building trade at Sheffield.")
for r in resolved:
    print(r)